[DIY Covid-19 Dashboard Kit](https://github.com/Yosefgid/Covid-19-Dashboard) (C) Yosef Gidey, 2023 ([ez18035@qmul.ac.uk](mailto:ez18035@qmul.ac.uk) - [web](http://www.eecs.qmul.ac.uk/~fabri/)). This notebook is released under the [GNU GPLv3.0 or later](https://www.gnu.org/licenses/).

# UK COVID-19 Cases VS Tests Dashboard

This dashboard will be displayed using [voila](https://voila.readthedocs.io/en/stable/index.html), a Python dashboarding tool that converts notebooks to standalone dashboards. 

This Dashboard shows the trends betweeen new infections(cases) and PCR tests, both by publishdate in England.

To view this dashboard template rendered in Voila click [here](https://mybinder.org/v2/gh/Yosefgid/Covid-19-Dashboard/HEAD?urlpath=%2Fvoila%2Frender%2FNewDashBoard.ipynb).

In [29]:
from IPython.display import clear_output
import ipywidgets as wdg
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from uk_covid19 import Cov19API

In [30]:
%matplotlib inline
# make figures larger
plt.rcParams['figure.dpi'] = 100

## Load initial data from disk

Initial "canned" data in ```casePerTest.json```. When the dashboard starts, it should load that data and assign it as a dictionary to the ```jsondata``` .

In [31]:
# Load JSON files and store the raw data in some variable. Edit as appropriate
with open("casePerTest.json", "rt") as INFILE:
    jsondata=json.load(INFILE)

## Wrangle the data

The dashboard wrangles the raw data into a ```DataFrame``` (or more than one, as required) that will be used for plotting. The wrangling code should be put into a function and called on the data from the JSON file (we'll need to call it again on any data downloaded from the API). The table here is not printed here, however can be printed if needed.

In [32]:
# Helper function for converting datastring to panda object
def convert_date(datestring):
        """ Convert a date string into a pandas datetime object """
        return pd.to_datetime(datestring, format="%Y-%m-%d")

In [33]:


def wrangle_data(rawdata):
    """ Parameters: rawdata - data from json file or API call. Returns a dataframe.
    Edit to include the code that wrangles the data, creates the dataframe and fills it in. """
    #wrangling the data
    datalist=rawdata['data']
    
    dates=[dictionary['date'] for dictionary in datalist ]
    dates.sort()
    
    startdate=convert_date(dates[0])
    enddate=convert_date(dates[-1])
    
    index=pd.date_range(startdate, enddate, freq='D')
    casePerTestdf=pd.DataFrame(index=index, columns=['cases', 'pcrTests'])

    for entry in datalist: # each entry is a dictionary with date, cases and tests
        date=convert_date(entry['date'])
        for column in ['cases', 'pcrTests']:
            # check that nothing is there yet - just in case some dates are duplicated,
            # maybe with data for different columns in each entry
            if pd.isna(casePerTestdf.loc[date, column]): 
                # replace None with 0 in our data 
                value= float(entry[column]) if entry[column]!=None else 0.0
                # this is the way you access a specific location in the dataframe - use .loc
                # and put index,column in a single set of [ ]
                casePerTestdf.loc[date, column]=value
            
    # fill in any remaining "holes" due to missing dates
    casePerTestdf.fillna(0.0, inplace=True)
    # Pickle file
    casePerTestdf.to_pickle("casePerTestdf.pkl")
    return(casePerTestdf)

# putting the wrangling code into a function allows you to call it again after refreshing the data through 
# the API. You should call the function directly on the JSON data when the dashboard starts, by including 
# the call in this cell as below:
casePerTestdf=wrangle_data(jsondata);

## Download current data

Here by clicking on the button below just generates some more random data and refreshes the graph. The button reads *Fetch Data*. This also retrives recent infromation.

In [34]:
# Place your API access code in this function. Do not call this function directly; it will be called by 
# the button callback. 
def access_api():
    """ Accesses the PHE API. Return data as a like-for-like replacement for the "canned" data loaded from the JSON file. """
    scope=[
    'areaType=nation',
    'areaName=England'
    ]
    metrics={
        "date": "date",
        "cases": "newCasesByPublishDate",
        "pcrTests": "newPCRTestsByPublishDate"
    }   
    api = Cov19API(filters=scope, structure=metrics)
    casePerTest=api.get_json()
    return casePerTest # return data read from the API

In [35]:
# Printout from this function will be lost in Voila unless captured in an
# output widget - therefore, we give feedback to the user by changing the 
# appearance of the button
def api_button_callback(button):
    """ Button callback - it must take the button as its parameter (unused in this case).
    Accesses API, wrangles data, updates global variable df used for plotting. """
    # Get fresh data from the API. If you have time, include some error handling
    # around this call.
    apidata=access_api()
    # wrangle the data and overwrite the dataframe for plotting
    global casePerTestdf
    casePerTestdf=wrangle_data(apidata)
    # the graph won't refresh until the user interacts with the widget.
    # this function simulates the interaction, see Graph and Analysis below.
    # The function needs to be adapted to your graph; you can omit this call
    # in the first instance
    refresh_graph()
    # after all is done, you can switch the icon on the button to a "check" sign
    # and optionally disable the button - it won't be needed again. If you are 
    # implementing error handling, you can use icons "unlink" or "times" and 
    # change the button text to "Unavailable" when the api call fails.
    apibutton.icon="check"
    apibutton.disabled=True

    
apibutton=wdg.Button(
    description='Fetch data', # you may want to change this...
    disabled=False,
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    tooltip="Fetch new PHE data",
    # FontAwesome names without the `fa-` prefix - try "download"
    icon=''
)
    
# remember to register your button callback function with the button
apibutton.on_click(api_button_callback) # the name of your function inside these brackets

display(apibutton)
# run all cells before clicking on this button

Button(button_style='info', description='Fetch data', style=ButtonStyle(), tooltip='Fetch new PHE data')

## Graphs and Analysis

The graph below shows the number of new cases and new PCR tests for England through time.

In [37]:
graphcols=wdg.SelectMultiple(
    options=['cases', 'pcrTests'],
    value=['cases', 'pcrTests'],
    rows=2,
    description='Option:',
    disabled=False
)

def plot_graph(graphcolumns):
    """ Our graph plotting function """
    numcols = len(graphcolumns)
    if numcols>0:
        casePerTestdf.plot(kind='line', y=list(graphcolumns))
        plt.show()
    else:
        print("Click to select data for graph")
        print("(CTRL-Click to select more than one category)")

def refresh_graph():
    """ We change the value of the widget in order to force a redraw of the graph;
    this is useful when the data have been updated. This is a bit of a gimmick; it
    needs to be customised for one of your widgets. """
    current=graphcols.value
    if current==graphcols.options[0]:
        other=graphcols.options[1]
    else:
        other=graphcols.options[0]
    graphcols.value=other # forces the redraw
    graphcols.value=current # now we can change it back
output=wdg.interactive_output(plot_graph, {'graphcolumns': graphcols})

display(graphcols, output)

SelectMultiple(description='Option:', index=(0, 1), options=('cases', 'pcrTests'), rows=2, value=('cases', 'pc…

Output()

**Author and Copyright Notice** Remember that if you deploy this dashboard as a Binder it will be publicly accessible. Take credit for your work! Also acknowledge your sources: Based on UK Government [data](https://coronavirus.data.gov.uk/) published by [Public Health England](https://www.gov.uk/government/organisations/public-health-england) and on the [DIY Covid Dashboard Kit](https://github.com/fsmeraldi/diy-covid19dash), Copyright (C) Fabrizio Smeraldi 2020,2023. Released under the [GNU GPLv3.0 or later](https://www.gnu.org/licenses/).